<a href="https://colab.research.google.com/github/davidofitaly/machine-learning-notes/blob/main/unsupervised/03_association_rules/02_apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Import of libraries

In [1]:
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x:.2f}')

In [2]:
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
!wget https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv

--2024-10-12 19:39:25--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/products.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.170.207, 173.194.174.207, 74.125.23.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.170.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2166953 (2.1M) [application/octet-stream]
Saving to: ‘products.csv’

products.csv        100%[===================>]   2.07M  1.07MB/s    in 1.9s    

2024-10-12 19:39:29 (1.07 MB/s) - ‘products.csv’ saved [2166953/2166953]

--2024-10-12 19:39:29--  https://storage.googleapis.com/esmartdata-courses-files/ml-course/orders.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.170.207, 173.194.174.207, 74.125.23.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.170.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24680147 (24M) [application/octet-stre

In [6]:
products = pd.read_csv('products.csv', usecols=['product_id', 'product_name'])
products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [7]:
orders = pd.read_csv('orders.csv', usecols=['order_id', 'product_id'])
orders

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633
...,...,...
1384612,3421063,14233
1384613,3421063,35548
1384614,3421070,35951
1384615,3421070,16953


###Data preparing

In [9]:
data = pd.merge(orders, products, how='inner', on='product_id', sort=True)
data = data.sort_values(by='order_id')
data.head()

,order_id,product_id,product_name
588447,1,22035,Organic Whole String Cheese
256931,1,10246,Organic Celery Hearts
277441,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese
1194893,1,43633,Lightly Smoked Sardines in Olive Oil
1302365,1,47209,Organic Hass Avocado


In [10]:
data.describe()

,order_id,product_id
count,1384617.00,1384617.00
mean,1706297.62,25556.24
std,989732.65,14121.27
min,1.00,1.00
25%,843370.00,13380.00
50%,1701880.00,25298.00
75%,2568023.00,37940.00
max,3421070.00,49688.00


In [12]:
data['product_name'].value_counts()

,count
product_name,
Banana,18726
Bag of Organic Bananas,15480
Organic Strawberries,10894
Organic Baby Spinach,9784
Large Lemon,8135
...,...
Original Raincoast Oat Crisp,1
Petite Creme Lowfat Strawberry Yogurt,1
Organic Petite Creme Mon Cherry Amour Yogurt,1


In [13]:
data['order_id'].nunique()

131209

In [15]:
transactions = data.groupby(by='order_id')['product_name'].apply(lambda name: ','.join(name))
transactions

,product_name
order_id,
1,"Organic Whole String Cheese,Organic Celery Hea..."
36,"Super Greens Salad,Spring Water,Organic Garnet..."
38,"Green Peas,Organic Baby Arugula,Shelled Pistac..."
96,"Organic Raspberries,Organic Pomegranate Kernel..."
98,"Uncured Applewood Smoked Bacon,Black Beans,Org..."
...,...
3421049,"Gluten Free Rice Bread,Organic Baby Broccoli,O..."
3421056,"Sparkling Lemon Water,Brioche Buns,Homestyle C..."
3421058,"Classic Britannia Crisps,Club Soda Lower Sodiu..."


In [16]:
transactions = transactions.str.split(',')
transactions

,product_name
order_id,
1,"[Organic Whole String Cheese, Organic Celery H..."
36,"[Super Greens Salad, Spring Water, Organic Gar..."
38,"[Green Peas, Organic Baby Arugula, Shelled Pis..."
96,"[Organic Raspberries, Organic Pomegranate Kern..."
98,"[Uncured Applewood Smoked Bacon, Black Beans, ..."
...,...
3421049,"[Gluten Free Rice Bread, Organic Baby Broccoli..."
3421056,"[Sparkling Lemon Water, Brioche Buns, Homestyl..."
3421058,"[Classic Britannia Crisps, Club Soda Lower Sod..."


###Code

In [17]:
from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()
encoder.fit(transactions)
transactions_encoded = encoder.transform(transactions, sparse=True)
transactions_encoded



<131209x40434 sparse matrix of type '<class 'numpy.bool_'>'
	with 1442410 stored elements in Compressed Sparse Row format>

In [18]:
transactions_encoded_df = pd.DataFrame(transactions_encoded.toarray(), columns=encoder.columns_)
transactions_encoded_df

,,Apricot & Banana Stage 2 Baby Food,Broad Spectrum SPF 30,Instant,Livermore Valley,Low Sodium Marinara,Premium,Vetiver scent,Whole,#2,...,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Island Berry Lime 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water,Lightly Seasoned with Rosemary and Roasted Garlic Family Size Herb Chicken Tortellini
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131204,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131205,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131206,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131207,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


###Apriori

In [22]:
from mlxtend.frequent_patterns import apriori, association_rules

supports = apriori(transactions_encoded_df, min_support=0.01, use_colnames=True)
supports = supports.sort_values(by='support', ascending=False)
supports.head()

,support,itemsets
8,0.14,(Banana)
7,0.12,(Bag of Organic Bananas)
76,0.08,(Organic Strawberries)
41,0.07,(Organic Baby Spinach)
31,0.06,(Large Lemon)


In [23]:
rules = association_rules(supports, metric='confidence', min_threshold=0)
rules = rules.iloc[:, [0, 1, 4, 5, 6]]
rules = rules.sort_values(by='lift', ascending=False)
rules.head(15)

,antecedents,consequents,support,confidence,lift
27,( Bag),(Clementines),0.01,0.79,36.84
26,(Clementines),( Bag),0.01,0.52,36.84
23,(Large Lemon),(Limes),0.01,0.20,4.26
22,(Limes),(Large Lemon),0.01,0.26,4.26
18,(Organic Raspberries),(Organic Strawberries),0.01,0.30,3.63
19,(Organic Strawberries),(Organic Raspberries),0.01,0.15,3.63
31,(Large Lemon),(Organic Avocado),0.01,0.17,2.94
30,(Organic Avocado),(Large Lemon),0.01,0.18,2.94
3,(Organic Hass Avocado),(Bag of Organic Bananas),0.02,0.33,2.81
2,(Bag of Organic Bananas),(Organic Hass Avocado),0.02,0.16,2.81
